In [1]:
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
from imgaug import augmenters as iaa
import random
import math

from keras.metrics import *
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from segmentation_models import Unet
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from segmentation_models.backbones import get_preprocessing

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/classification_models/resnext/__init__.py:4: UserWarning: Current ResNext models are deprecated, use keras.applications ResNeXt models
  warnings.warn('Current ResNext models are deprecated, '


In [2]:
train_frame_path = '/mnt/data/aja/diplomka/disk/train_imgs/train/'
train_mask_path = '/mnt/data/aja/diplomka/disk/train_masks/train'

val_frame_path = '/mnt/data/aja/diplomka/disk/val_imgs/val'
val_mask_path = '/mnt/data/aja/diplomka/disk/val_masks/val'

img_height = 512
img_width = 512
img_channels = 3

no_train_imgs = len(os.listdir(train_frame_path))
no_val_imgs = len(os.listdir(val_frame_path))

epochs = 50
batch_size = 2

In [3]:
def augmentation():
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    seq = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.Crop(percent=(0, 0.1)), 
        iaa.GaussianBlur((0, 1.0)), 
        iaa.ContrastNormalization((0.9, 1.1)),
        iaa.Multiply((0.9, 1.1), per_channel=0.1),
        sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1))),
        iaa.Affine(
            scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
            translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
            rotate=(-0.01, 0.01),
            shear=(-8, 8)
        )
    ], random_order=True) 
    return seq

seq = augmentation()

In [4]:
def data_gen(path_to_imgs, path_to_masks, batch_size, preprocess=False):
    names = os.listdir(path_to_imgs)
    random.shuffle(names)
    while 1:
        img = []
        mask = []       
        all_aug_im = []
        all_aug_ma = []
        
        for idx in range(int(batch_size/2)):
#             train_img = image_ut.normalized(cv2.imread(path_to_imgs+'/'+names[idx]))#/255
            train_img = cv2.imread(path_to_imgs+'/'+names[idx])
            train_img =  cv2.resize(train_img, (img_height, img_width))
            img.append(train_img)
            
            train_mask = cv2.imread(path_to_masks+'/'+names[idx], cv2.IMREAD_GRAYSCALE)#/255
            train_mask = cv2.resize(train_mask, (img_height, img_width))
            train_mask[train_mask > 50] = 255
            train_mask[train_mask <= 50] = 0
            train_mask = train_mask.reshape(img_height, img_width, 1)
            mask.append(train_mask)
        
        for i in range(5):
            seq = augmentation()
            seq_det = seq.to_deterministic()
            images_aug = seq_det.augment_images(np.array(img).astype(np.uint8))    
            mask_aug = seq_det.augment_images(np.array(mask).astype(np.uint8))
            mask_aug[mask_aug > 50] = 255
            mask_aug[mask_aug <= 50] = 0
            for im in range(len(images_aug)):
                all_aug_im.append(images_aug[im])
                all_aug_ma.append(mask_aug[im])
        
        for idx in range(len(all_aug_im)):
            img.append(all_aug_im[idx])
            mask.append(all_aug_ma[idx])
        if preprocess:
            imgs = [preprocess_input(x)/255. for x in img]
        else:
            imgs = [x/255. for x in img]
        masks = [y/255. for y in mask]
        
        imgs_shuffled = []
        masks_shuffled = []
        shuffled_indices = np.arange(len(imgs))
        random.shuffle(shuffled_indices)
        for ix in shuffled_indices:
            imgs_shuffled.append(imgs[ix])
            masks_shuffled.append(masks[ix])

        cnt_im = math.floor(len(imgs_shuffled)/batch_size)
        inkr = 0
        for i in range(cnt_im):
            start = 0+inkr
            stop = 0+inkr+batch_size
            batch_imgs = imgs_shuffled[start:stop]
            batch_masks = masks_shuffled[start:stop]
            inkr = inkr + batch_size
            yield np.array(batch_imgs), np.array(batch_masks)

In [5]:
def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

# Adam Resnet 18

In [6]:
backbone = 'resnet18'
preprocess_input = get_preprocessing(backbone)

train_gen = data_gen(train_frame_path, train_mask_path, batch_size, preprocess=True)
val_gen = data_gen(val_frame_path, val_mask_path, batch_size, preprocess=True)

model = Unet(backbone, classes=1, activation='sigmoid', encoder_weights='imagenet')
model.compile('Adam', loss=dice_coef_loss, metrics=["accuracy", dice_coef])
model.summary()


earlystopper = EarlyStopping(patience=10, verbose=1)
reducelronplateau = ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1)
checkpointer = ModelCheckpoint('model_unet_disks.h5', verbose=1, save_best_only=True)
callbacks_list = [checkpointer, reducelronplateau, earlystopper]

results = model.fit_generator(train_gen, epochs=100, 
                              steps_per_epoch = (no_train_imgs*6//batch_size),
                              validation_data=val_gen, 
                              validation_steps=(no_val_imgs*6//batch_size), 
#                               callbacks=callbacks_list,
                              verbose=1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, None, None, 3 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, None, None, 3 9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn0 (Batch

Epoch 1/100
372/372 [==============================] - 70s 187ms/step - loss: 0.0096 - acc: 0.9886 - dice_coef: 0.9904 - val_loss: 0.1479 - val_acc: 0.6970 - val_dice_coef: 0.8521
Epoch 2/100
372/372 [==============================] - 59s 158ms/step - loss: 0.0024 - acc: 0.9944 - dice_coef: 0.9976 - val_loss: 0.2233 - val_acc: 0.5437 - val_dice_coef: 0.7767
Epoch 3/100
249/372 [===================>..........] - ETA: 17s - loss: 0.0017 - acc: 0.9960 - dice_coef: 0.9983

KeyboardInterrupt: 

# Adam resnet 101

In [15]:
backbone = 'resnet101'
preprocess_input = get_preprocessing(backbone)

train_gen = data_gen(train_frame_path, train_mask_path, batch_size, preprocess=True)
val_gen = data_gen(val_frame_path, val_mask_path, batch_size, preprocess=True)

model = Unet(backbone, classes=1, activation='sigmoid', encoder_weights='imagenet')
model.compile('Adam', loss=dice_coef_loss, metrics=["accuracy", dice_coef])
model.summary()


earlystopper = EarlyStopping(patience=10, verbose=1)
reducelronplateau = ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1)
checkpointer = ModelCheckpoint('model_unet_disks.h5', verbose=1, save_best_only=True)
callbacks_list = [checkpointer, reducelronplateau, earlystopper]

results = model.fit_generator(train_gen, epochs=epochs, 
                              steps_per_epoch = (no_train_imgs*6//batch_size),
                              validation_data=val_gen, 
                              validation_steps=(no_val_imgs*6//batch_size), 
                              callbacks=callbacks_list,
                              verbose=1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, None, None, 3 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, None, None, 3 9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_36 (ZeroPadding2 (None, None, None, 3 0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, None, None, 6 9408        zero_padding2d_36[0][0]          
__________________________________________________________________________________________________
bn0 (Batch

Epoch 1/50
372/372 [==============================] - 196s 527ms/step - loss: 0.0079 - acc: 0.9851 - dice_coef: 0.9921 - val_loss: 0.0585 - val_acc: 0.8785 - val_dice_coef: 0.9415

Epoch 00001: val_loss improved from inf to 0.05851, saving model to model_unet_disks.h5
Epoch 2/50
372/372 [==============================] - 157s 423ms/step - loss: 0.0030 - acc: 0.9928 - dice_coef: 0.9970 - val_loss: 0.0718 - val_acc: 0.8510 - val_dice_coef: 0.9282

Epoch 00002: val_loss did not improve from 0.05851
Epoch 3/50
372/372 [==============================] - 158s 425ms/step - loss: 0.0025 - acc: 0.9939 - dice_coef: 0.9975 - val_loss: 0.0354 - val_acc: 0.9236 - val_dice_coef: 0.9646

Epoch 00003: val_loss improved from 0.05851 to 0.03539, saving model to model_unet_disks.h5
Epoch 4/50
372/372 [==============================] - 158s 424ms/step - loss: 9.8748e-04 - acc: 0.9976 - dice_coef: 0.9990 - val_loss: 0.0773 - val_acc: 0.8433 - val_dice_coef: 0.9227

Epoch 00004: val_loss did not improve fro

In [16]:
# serialize model to JSON
model_json = model.to_json()
with open("models/model_unet_disk_pretrainedRESNET101.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/model_unet_disk_pretrainedRESNET101.h5")
print("Saved model to disk")

Saved model to disk


# Adam RESNET 152 

In [17]:
backbone = 'resnet152'
preprocess_input = get_preprocessing(backbone)

train_gen = data_gen(train_frame_path, train_mask_path, batch_size, preprocess=True)
val_gen = data_gen(val_frame_path, val_mask_path, batch_size, preprocess=True)

model = Unet(backbone, classes=1, activation='sigmoid', encoder_weights='imagenet')
model.compile('Adam', loss=dice_coef_loss, metrics=["accuracy", dice_coef])
model.summary()


earlystopper = EarlyStopping(patience=10, verbose=1)
reducelronplateau = ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1)
checkpointer = ModelCheckpoint('model_unet_disks.h5', verbose=1, save_best_only=True)
callbacks_list = [checkpointer, reducelronplateau, earlystopper]

results = model.fit_generator(train_gen, epochs=epochs, 
                              steps_per_epoch = (no_train_imgs*6//batch_size),
                              validation_data=val_gen, 
                              validation_steps=(no_val_imgs*6//batch_size), 
                              callbacks=callbacks_list,
                              verbose=1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, None, None, 3 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, None, None, 3 9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_71 (ZeroPadding2 (None, None, None, 3 0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, None, None, 6 9408        zero_padding2d_71[0][0]          
__________________________________________________________________________________________________
bn0 (Batch

add_106 (Add)                   (None, None, None, 1 0           stage3_unit29_conv3[0][0]        
                                                                 add_105[0][0]                    
__________________________________________________________________________________________________
stage3_unit30_bn1 (BatchNormali (None, None, None, 1 4096        add_106[0][0]                    
__________________________________________________________________________________________________
stage3_unit30_relu1 (Activation (None, None, None, 1 0           stage3_unit30_bn1[0][0]          
__________________________________________________________________________________________________
stage3_unit30_conv1 (Conv2D)    (None, None, None, 2 262144      stage3_unit30_relu1[0][0]        
__________________________________________________________________________________________________
stage3_unit30_bn2 (BatchNormali (None, None, None, 2 1024        stage3_unit30_conv1[0][0]        
__________

Epoch 1/50
372/372 [==============================] - 266s 714ms/step - loss: 0.0069 - acc: 0.9873 - dice_coef: 0.9931 - val_loss: 0.3636 - val_acc: 0.2686 - val_dice_coef: 0.6364

Epoch 00001: val_loss improved from inf to 0.36359, saving model to model_unet_disks.h5
Epoch 2/50
372/372 [==============================] - 205s 551ms/step - loss: 0.0033 - acc: 0.9920 - dice_coef: 0.9967 - val_loss: 0.3359 - val_acc: 0.3206 - val_dice_coef: 0.6641

Epoch 00002: val_loss improved from 0.36359 to 0.33595, saving model to model_unet_disks.h5
Epoch 3/50
372/372 [==============================] - 206s 553ms/step - loss: 0.0023 - acc: 0.9943 - dice_coef: 0.9977 - val_loss: 0.1568 - val_acc: 0.6690 - val_dice_coef: 0.8432

Epoch 00003: val_loss improved from 0.33595 to 0.15679, saving model to model_unet_disks.h5
Epoch 4/50
372/372 [==============================] - 206s 553ms/step - loss: 0.0016 - acc: 0.9962 - dice_coef: 0.9984 - val_loss: 0.1441 - val_acc: 0.7009 - val_dice_coef: 0.8559

Epoc

In [18]:
# serialize model to JSON
model_json = model.to_json()
with open("models/model_unet_disk_pretrainedRESNET152.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/model_unet_disk_pretrainedRESNET152.h5")
print("Saved model to disk")

Saved model to disk


# Adam Inception Resnet

In [6]:
backbone = 'inceptionresnetv2'
preprocess_input = get_preprocessing(backbone)

train_gen = data_gen(train_frame_path, train_mask_path, batch_size, preprocess=False)
val_gen = data_gen(val_frame_path, val_mask_path, batch_size, preprocess=False)

model = Unet(backbone, classes=1, activation='sigmoid', encoder_weights='imagenet')
model.compile('Adam', loss=dice_coef_loss, metrics=["accuracy", dice_coef])
model.summary()


earlystopper = EarlyStopping(patience=10, verbose=1)
reducelronplateau = ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1)
checkpointer = ModelCheckpoint('model_unet_disks.h5', verbose=1, save_best_only=True)
callbacks_list = [checkpointer, reducelronplateau, earlystopper]

results = model.fit_generator(train_gen, epochs=epochs, 
                              steps_per_epoch = (no_train_imgs*6//batch_size),
                              validation_data=val_gen, 
                              validation_steps=(no_val_imgs*6//batch_size), 
                              callbacks=callbacks_list,
                              verbose=1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

conv2d_113 (Conv2D)             (None, None, None, 1 208896      block17_9_ac[0][0]               
__________________________________________________________________________________________________
conv2d_116 (Conv2D)             (None, None, None, 1 215040      activation_115[0][0]             
__________________________________________________________________________________________________
batch_normalization_113 (BatchN (None, None, None, 1 576         conv2d_113[0][0]                 
__________________________________________________________________________________________________
batch_normalization_116 (BatchN (None, None, None, 1 576         conv2d_116[0][0]                 
__________________________________________________________________________________________________
activation_113 (Activation)     (None, None, None, 1 0           batch_normalization_113[0][0]    
__________________________________________________________________________________________________
activation

Epoch 1/50
372/372 [==============================] - 261s 701ms/step - loss: 0.0060 - acc: 0.9911 - dice_coef: 0.9940 - val_loss: 0.0811 - val_acc: 0.8304 - val_dice_coef: 0.9189

Epoch 00001: val_loss improved from inf to 0.08113, saving model to model_unet_disks.h5
Epoch 2/50
372/372 [==============================] - 202s 544ms/step - loss: 0.0016 - acc: 0.9963 - dice_coef: 0.9984 - val_loss: 0.0779 - val_acc: 0.8310 - val_dice_coef: 0.9221

Epoch 00002: val_loss improved from 0.08113 to 0.07795, saving model to model_unet_disks.h5
Epoch 3/50
372/372 [==============================] - 202s 544ms/step - loss: 8.5493e-04 - acc: 0.9980 - dice_coef: 0.9991 - val_loss: 0.0670 - val_acc: 0.8592 - val_dice_coef: 0.9330

Epoch 00003: val_loss improved from 0.07795 to 0.06697, saving model to model_unet_disks.h5
Epoch 4/50
372/372 [==============================] - 203s 545ms/step - loss: 5.6924e-04 - acc: 0.9986 - dice_coef: 0.9994 - val_loss: 0.0834 - val_acc: 0.8265 - val_dice_coef: 0.91

In [7]:
# serialize model to JSON
model_json = model.to_json()
with open("models/model_unet_disk_pretrainedINCEPTION_RESNET.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/model_unet_disk_pretrainedINCEPTION_RESNET.h5")
print("Saved model to disk")

Saved model to disk


# Adam Inception 

In [8]:
backbone = 'inceptionv3'
preprocess_input = get_preprocessing(backbone)

train_gen = data_gen(train_frame_path, train_mask_path, batch_size, preprocess=False)
val_gen = data_gen(val_frame_path, val_mask_path, batch_size, preprocess=False)

model = Unet(backbone, classes=1, activation='sigmoid', encoder_weights='imagenet')
model.compile('Adam', loss=dice_coef_loss, metrics=["accuracy", dice_coef])
model.summary()


earlystopper = EarlyStopping(patience=10, verbose=1)
reducelronplateau = ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1)
checkpointer = ModelCheckpoint('model_unet_disks.h5', verbose=1, save_best_only=True)
callbacks_list = [checkpointer, reducelronplateau, earlystopper]

results = model.fit_generator(train_gen, epochs=epochs, 
                              steps_per_epoch = (no_train_imgs*6//batch_size),
                              validation_data=val_gen, 
                              validation_steps=(no_val_imgs*6//batch_size), 
                              callbacks=callbacks_list,
                              verbose=1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_204 (Conv2D)             (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_204 (BatchN (None, None, None, 3 96          conv2d_204[0][0]                 
__________________________________________________________________________________________________
activation_204 (Activation)     (None, None, None, 3 0           batch_normalization_204[0][0]    
__________________________________________________________________________________________________
conv2d_205

Epoch 1/50
372/372 [==============================] - 147s 394ms/step - loss: 0.0062 - acc: 0.9896 - dice_coef: 0.9938 - val_loss: 0.2910 - val_acc: 0.3875 - val_dice_coef: 0.7090

Epoch 00001: val_loss improved from inf to 0.29100, saving model to model_unet_disks.h5
Epoch 2/50
372/372 [==============================] - 110s 296ms/step - loss: 0.0025 - acc: 0.9944 - dice_coef: 0.9975 - val_loss: 0.3451 - val_acc: 0.2959 - val_dice_coef: 0.6549

Epoch 00002: val_loss did not improve from 0.29100
Epoch 3/50
372/372 [==============================] - 111s 299ms/step - loss: 0.0017 - acc: 0.9961 - dice_coef: 0.9983 - val_loss: 0.2417 - val_acc: 0.4957 - val_dice_coef: 0.7583

Epoch 00003: val_loss improved from 0.29100 to 0.24166, saving model to model_unet_disks.h5
Epoch 4/50
372/372 [==============================] - 112s 301ms/step - loss: 7.4498e-04 - acc: 0.9982 - dice_coef: 0.9993 - val_loss: 0.2437 - val_acc: 0.4945 - val_dice_coef: 0.7563

Epoch 00004: val_loss did not improve fro

KeyboardInterrupt: 

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("models/model_unet_disk_pretrainedINCEPTION.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/model_unet_disk_pretrainedINCEPTION.h5")
print("Saved model to disk")

# SGD resnet18

In [23]:
backbone = 'resnet18'
preprocess_input = get_preprocessing(backbone)

train_gen = data_gen(train_frame_path, train_mask_path, batch_size, preprocess=True)
val_gen = data_gen(val_frame_path, val_mask_path, batch_size, preprocess=True)

model = Unet(backbone, classes=1, activation='sigmoid', encoder_weights='imagenet')
model.compile('SGD', loss=dice_coef_loss, metrics=["accuracy", dice_coef])
model.summary()


earlystopper = EarlyStopping(patience=10, verbose=1)
reducelronplateau = ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1)
checkpointer = ModelCheckpoint('model_unet_disks.h5', verbose=1, save_best_only=True)
callbacks_list = [checkpointer, reducelronplateau, earlystopper]

results = model.fit_generator(train_gen, epochs=epochs, 
                              steps_per_epoch = (no_train_imgs*6//batch_size),
                              validation_data=val_gen, 
                              validation_steps=(no_val_imgs*6//batch_size), 
                              callbacks=callbacks_list,
                              verbose=1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, None, None, 3 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, None, None, 3 9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_123 (ZeroPadding (None, None, None, 3 0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, None, None, 6 9408        zero_padding2d_123[0][0]         
__________________________________________________________________________________________________
bn0 (Batch

Epoch 1/50
372/372 [==============================] - 120s 324ms/step - loss: 0.0361 - acc: 0.9370 - dice_coef: 0.9639 - val_loss: 0.0711 - val_acc: 0.8369 - val_dice_coef: 0.9289

Epoch 00001: val_loss improved from inf to 0.07107, saving model to model_unet_disks.h5
Epoch 2/50
372/372 [==============================] - 59s 159ms/step - loss: 0.0111 - acc: 0.9801 - dice_coef: 0.9889 - val_loss: 0.0605 - val_acc: 0.8557 - val_dice_coef: 0.9395

Epoch 00002: val_loss improved from 0.07107 to 0.06053, saving model to model_unet_disks.h5
Epoch 3/50
372/372 [==============================] - 60s 161ms/step - loss: 0.0082 - acc: 0.9839 - dice_coef: 0.9918 - val_loss: 0.0604 - val_acc: 0.8542 - val_dice_coef: 0.9396

Epoch 00003: val_loss improved from 0.06053 to 0.06043, saving model to model_unet_disks.h5
Epoch 4/50
372/372 [==============================] - 60s 161ms/step - loss: 0.0069 - acc: 0.9858 - dice_coef: 0.9931 - val_loss: 0.0599 - val_acc: 0.8558 - val_dice_coef: 0.9401

Epoch 0

In [24]:
# serialize model to JSON
model_json = model.to_json()
with open("models/model_unet_disk_pretrainedRESNET18_SGD.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/model_unet_disk_pretrainedRESNET18_SGD.h5")
print("Saved model to disk")

Saved model to disk


# SGD resnet101

In [25]:
backbone = 'resnet101'
preprocess_input = get_preprocessing(backbone)

train_gen = data_gen(train_frame_path, train_mask_path, batch_size, preprocess=True)
val_gen = data_gen(val_frame_path, val_mask_path, batch_size, preprocess=True)

model = Unet(backbone, classes=1, activation='sigmoid', encoder_weights='imagenet')
model.compile('SGD', loss=dice_coef_loss, metrics=["accuracy", dice_coef])
model.summary()


earlystopper = EarlyStopping(patience=10, verbose=1)
reducelronplateau = ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1)
checkpointer = ModelCheckpoint('model_unet_disks.h5', verbose=1, save_best_only=True)
callbacks_list = [checkpointer, reducelronplateau, earlystopper]

results = model.fit_generator(train_gen, epochs=epochs, 
                              steps_per_epoch = (no_train_imgs*6//batch_size),
                              validation_data=val_gen, 
                              validation_steps=(no_val_imgs*6//batch_size), 
                              callbacks=callbacks_list,
                              verbose=1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, None, None, 3 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, None, None, 3 9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_141 (ZeroPadding (None, None, None, 3 0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, None, None, 6 9408        zero_padding2d_141[0][0]         
__________________________________________________________________________________________________
bn0 (Batch

Total params: 51,605,466
Trainable params: 51,505,684
Non-trainable params: 99,782
__________________________________________________________________________________________________
Epoch 1/50
372/372 [==============================] - 223s 598ms/step - loss: 0.0446 - acc: 0.9276 - dice_coef: 0.9554 - val_loss: 0.0484 - val_acc: 0.8821 - val_dice_coef: 0.9516

Epoch 00001: val_loss improved from inf to 0.04836, saving model to model_unet_disks.h5
Epoch 2/50
372/372 [==============================] - 146s 392ms/step - loss: 0.0123 - acc: 0.9778 - dice_coef: 0.9877 - val_loss: 0.0490 - val_acc: 0.8768 - val_dice_coef: 0.9510

Epoch 00002: val_loss did not improve from 0.04836
Epoch 3/50
372/372 [==============================] - 147s 394ms/step - loss: 0.0091 - acc: 0.9820 - dice_coef: 0.9909 - val_loss: 0.0508 - val_acc: 0.8726 - val_dice_coef: 0.9492

Epoch 00003: val_loss did not improve from 0.04836
Epoch 4/50
372/372 [==============================] - 147s 394ms/step - loss: 0.0071 

In [26]:
# serialize model to JSON
model_json = model.to_json()
with open("models/model_unet_disk_pretrainedRESNET101_SGD.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/model_unet_disk_pretrainedRESNET101_SGD.h5")
print("Saved model to disk")

Saved model to disk


# SGD resnet152

In [27]:
backbone = 'resnet152'
preprocess_input = get_preprocessing(backbone)

train_gen = data_gen(train_frame_path, train_mask_path, batch_size, preprocess=True)
val_gen = data_gen(val_frame_path, val_mask_path, batch_size, preprocess=True)

model = Unet(backbone, classes=1, activation='sigmoid', encoder_weights='imagenet')
model.compile('SGD', loss=dice_coef_loss, metrics=["accuracy", dice_coef])
model.summary()


earlystopper = EarlyStopping(patience=10, verbose=1)
reducelronplateau = ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1)
checkpointer = ModelCheckpoint('model_unet_disks.h5', verbose=1, save_best_only=True)
callbacks_list = [checkpointer, reducelronplateau, earlystopper]

results = model.fit_generator(train_gen, epochs=epochs, 
                              steps_per_epoch = (no_train_imgs*6//batch_size),
                              validation_data=val_gen, 
                              validation_steps=(no_val_imgs*6//batch_size), 
                              callbacks=callbacks_list,
                              verbose=1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, None, None, 3 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, None, None, 3 9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_176 (ZeroPadding (None, None, None, 3 0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, None, None, 6 9408        zero_padding2d_176[0][0]         
__________________________________________________________________________________________________
bn0 (Batch

zero_padding2d_220 (ZeroPadding (None, None, None, 2 0           stage3_unit32_relu2[0][0]        
__________________________________________________________________________________________________
stage3_unit32_conv2 (Conv2D)    (None, None, None, 2 589824      zero_padding2d_220[0][0]         
__________________________________________________________________________________________________
stage3_unit32_bn3 (BatchNormali (None, None, None, 2 1024        stage3_unit32_conv2[0][0]        
__________________________________________________________________________________________________
stage3_unit32_relu3 (Activation (None, None, None, 2 0           stage3_unit32_bn3[0][0]          
__________________________________________________________________________________________________
stage3_unit32_conv3 (Conv2D)    (None, None, None, 1 262144      stage3_unit32_relu3[0][0]        
__________________________________________________________________________________________________
add_200 (A

Epoch 1/50
372/372 [==============================] - 293s 787ms/step - loss: 0.0392 - acc: 0.9392 - dice_coef: 0.9608 - val_loss: 0.0533 - val_acc: 0.8620 - val_dice_coef: 0.9467

Epoch 00001: val_loss improved from inf to 0.05326, saving model to model_unet_disks.h5
Epoch 2/50
372/372 [==============================] - 189s 509ms/step - loss: 0.0101 - acc: 0.9823 - dice_coef: 0.9899 - val_loss: 0.0234 - val_acc: 0.9486 - val_dice_coef: 0.9766

Epoch 00002: val_loss improved from 0.05326 to 0.02338, saving model to model_unet_disks.h5
Epoch 3/50
372/372 [==============================] - 189s 509ms/step - loss: 0.0073 - acc: 0.9857 - dice_coef: 0.9927 - val_loss: 0.0303 - val_acc: 0.9280 - val_dice_coef: 0.9697

Epoch 00003: val_loss did not improve from 0.02338
Epoch 4/50
372/372 [==============================] - 190s 510ms/step - loss: 0.0059 - acc: 0.9880 - dice_coef: 0.9941 - val_loss: 0.0232 - val_acc: 0.9460 - val_dice_coef: 0.9768

Epoch 00004: val_loss improved from 0.02338 t

In [28]:
# serialize model to JSON
model_json = model.to_json()
with open("models/model_unet_disk_pretrainedRESNET152_SGD.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/model_unet_disk_pretrainedRESNET152_SGD.h5")
print("Saved model to disk")

Saved model to disk


# SGD inception_resnet

In [29]:
backbone = 'inceptionresnetv2'
preprocess_input = get_preprocessing(backbone)

train_gen = data_gen(train_frame_path, train_mask_path, batch_size, preprocess=True)
val_gen = data_gen(val_frame_path, val_mask_path, batch_size, preprocess=True)

model = Unet(backbone, classes=1, activation='sigmoid', encoder_weights='imagenet')
model.compile('SGD', loss=dice_coef_loss, metrics=["accuracy", dice_coef])
model.summary()


earlystopper = EarlyStopping(patience=10, verbose=1)
reducelronplateau = ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1)
checkpointer = ModelCheckpoint('model_unet_disks.h5', verbose=1, save_best_only=True)
callbacks_list = [checkpointer, reducelronplateau, earlystopper]

results = model.fit_generator(train_gen, epochs=epochs, 
                              steps_per_epoch = (no_train_imgs*6//batch_size),
                              validation_data=val_gen, 
                              validation_steps=(no_val_imgs*6//batch_size), 
                              callbacks=callbacks_list,
                              verbose=1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_298 (Conv2D)             (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_298 (BatchN (None, None, None, 3 96          conv2d_298[0][0]                 
__________________________________________________________________________________________________
activation_298 (Activation)     (None, None, None, 3 0           batch_normalization_298[0][0]    
__________________________________________________________________________________________________
conv2d_299

block17_12 (Lambda)             (None, None, None, 1 0           block17_11_ac[0][0]              
                                                                 block17_12_conv[0][0]            
__________________________________________________________________________________________________
block17_12_ac (Activation)      (None, None, None, 1 0           block17_12[0][0]                 
__________________________________________________________________________________________________
conv2d_423 (Conv2D)             (None, None, None, 1 139264      block17_12_ac[0][0]              
__________________________________________________________________________________________________
batch_normalization_423 (BatchN (None, None, None, 1 384         conv2d_423[0][0]                 
__________________________________________________________________________________________________
activation_423 (Activation)     (None, None, None, 1 0           batch_normalization_423[0][0]    
__________

Epoch 1/50
372/372 [==============================] - 319s 858ms/step - loss: 0.0663 - acc: 0.8699 - dice_coef: 0.9337 - val_loss: 0.0507 - val_acc: 0.9007 - val_dice_coef: 0.9493

Epoch 00001: val_loss improved from inf to 0.05070, saving model to model_unet_disks.h5
Epoch 2/50
372/372 [==============================] - 187s 503ms/step - loss: 0.0130 - acc: 0.9788 - dice_coef: 0.9870 - val_loss: 0.0452 - val_acc: 0.8994 - val_dice_coef: 0.9548

Epoch 00002: val_loss improved from 0.05070 to 0.04516, saving model to model_unet_disks.h5
Epoch 3/50
372/372 [==============================] - 187s 503ms/step - loss: 0.0084 - acc: 0.9842 - dice_coef: 0.9916 - val_loss: 0.0394 - val_acc: 0.9104 - val_dice_coef: 0.9606

Epoch 00003: val_loss improved from 0.04516 to 0.03943, saving model to model_unet_disks.h5
Epoch 4/50
372/372 [==============================] - 188s 504ms/step - loss: 0.0064 - acc: 0.9873 - dice_coef: 0.9936 - val_loss: 0.0429 - val_acc: 0.9019 - val_dice_coef: 0.9571

Epoc

In [30]:
# serialize model to JSON
model_json = model.to_json()
with open("models/model_unet_disk_pretrainedINC_RESNET_SGD.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/model_unet_disk_pretrainedINC_RESNET_SGD.h5")
print("Saved model to disk")

Saved model to disk


# SGD inception

In [31]:
backbone = 'inceptionv3'
preprocess_input = get_preprocessing(backbone)

train_gen = data_gen(train_frame_path, train_mask_path, batch_size, preprocess=True)
val_gen = data_gen(val_frame_path, val_mask_path, batch_size, preprocess=True)

model = Unet(backbone, classes=1, activation='sigmoid', encoder_weights='imagenet')
model.compile('SGD', loss=dice_coef_loss, metrics=["accuracy", dice_coef])
model.summary()


earlystopper = EarlyStopping(patience=10, verbose=1)
reducelronplateau = ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1)
checkpointer = ModelCheckpoint('model_unet_disks.h5', verbose=1, save_best_only=True)
callbacks_list = [checkpointer, reducelronplateau, earlystopper]

results = model.fit_generator(train_gen, epochs=epochs, 
                              steps_per_epoch = (no_train_imgs*6//batch_size),
                              validation_data=val_gen, 
                              validation_steps=(no_val_imgs*6//batch_size), 
                              callbacks=callbacks_list,
                              verbose=1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_501 (Conv2D)             (None, None, None, 3 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_501 (BatchN (None, None, None, 3 96          conv2d_501[0][0]                 
__________________________________________________________________________________________________
activation_501 (Activation)     (None, None, None, 3 0           batch_normalization_501[0][0]    
__________________________________________________________________________________________________
conv2d_502

Epoch 1/50
372/372 [==============================] - 230s 617ms/step - loss: 0.0517 - acc: 0.9092 - dice_coef: 0.9483 - val_loss: 0.0543 - val_acc: 0.8723 - val_dice_coef: 0.9457

Epoch 00001: val_loss improved from inf to 0.05431, saving model to model_unet_disks.h5
Epoch 2/50
372/372 [==============================] - 105s 281ms/step - loss: 0.0114 - acc: 0.9819 - dice_coef: 0.9886 - val_loss: 0.0494 - val_acc: 0.8797 - val_dice_coef: 0.9506

Epoch 00002: val_loss improved from 0.05431 to 0.04941, saving model to model_unet_disks.h5
Epoch 3/50
372/372 [==============================] - 106s 286ms/step - loss: 0.0070 - acc: 0.9871 - dice_coef: 0.9930 - val_loss: 0.0431 - val_acc: 0.8964 - val_dice_coef: 0.9569

Epoch 00003: val_loss improved from 0.04941 to 0.04306, saving model to model_unet_disks.h5
Epoch 4/50
372/372 [==============================] - 107s 287ms/step - loss: 0.0053 - acc: 0.9897 - dice_coef: 0.9947 - val_loss: 0.0373 - val_acc: 0.9118 - val_dice_coef: 0.9627

Epoc

In [32]:
# serialize model to JSON
model_json = model.to_json()
with open("models/model_unet_disk_pretrainedINCEPTION_SGD.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/model_unet_disk_pretrainedINCEPTION_SGD.h5")
print("Saved model to disk")

Saved model to disk
